<a href="https://colab.research.google.com/github/ckkissane/deep_learning_curriculum/blob/master/solutions/1_Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Implement a decoder-only transformer language model.

Here are some first principle questions to answer:

## What is different architecturally from the Transformer, vs a normal RNN, like an LSTM? (Specifically, how are recurrence and time managed?)

Transformer:
* Non sequential: sequences are processed as a whole using multi-headed attention layers, which allows for parallel computation
* Positional encodings are used so that the transformer can capture sequential information

RNN:
* Sequential processing: sequences are processed one token at a time using recurrent layers, which is not parallelizable
* No positional encoding: RNNs learn positional information based on the past hidden state. This can cause issues with long sequences, as we lose information from older inputs

## Attention is defined as, Attention(Q,K,V) = softmax(QK^T/sqrt(d_k))V. What are the dimensions for Q, K, and V? Why do we use this setup? What other combinations could we do with (Q,K) that also output weights?

The dimensions are:
* Q: (seq_len, d_k)
* K: (seq_len, d_k)
* V: (seq_len, d_v)

1. d_k represents the dimension of the vectors representing the queries / keys. 
2. d_v is the dimension of the vectors representing the values.
3. Since there are query, key, and value vectors for each token in the sequence, it's natural to pack them into matrices for more efficient computation. That's why we have seq_len rows for each matrix. 


Other combinations we could do with (Q, K) that output weights:
* Additive attention computes the compatibility function using a feed-forward network with a single hidden layer

However, "dot-product attention is
much faster and more space-efficient in practice, since it can be implemented using highly optimized
matrix multiplication code."

## Are the dense layers different at each multi-head attention block? Why or why not?

Yes

Here are some ideas why:
* Intuitively, the point of stacking layers is so that each layer can transform the data independently of each other, resulting in a more expressive model
* The W^Q, W^K, W^V layers learn representations for the query, key, and values. The model will likely benefit from the flexibility of learning different representations for each block
* It's been empirically observed that [more parameters lead to better performance](https://arxiv.org/abs/2001.08361)

## Why do we have so many skip connections, especially connecting the input of an attention function to the output? Intuitively, what if we didn't?

In the [ResNet paper](https://arxiv.org/abs/1512.03385?context=cs), it was observed that some deep neural networks perform worse than their shallow counterparts. Adding skip connections empirically seemed to solve this issue. 
The intuition is that adding skip connections allows layers to learn the identity mapping more easily. 
"To the extreme, if an identity mapping were optimal, it would be easier to push the residual to zero than to fit an identity mapping by a stack of nonlinear layers."

If we didn't include these skip connections, we might experience a degradation of performance for very deep transformer models due to vanishing / exploding gradient problems.

## Now we'll actually implement the code. Make sure each of these is completely correct - it's very easy to get the small details wrong. Implement the positional embedding function first.

In [1]:
import torch
import torch.nn.functional as F
from torch import optim
from torch import nn
from torch import einsum
from torch.utils.data import Dataset
from torch.utils.data.dataloader import DataLoader
import random
import numpy as np
import math
import copy
from tqdm import tqdm
import re

In [2]:
# I use learned encodings, rather than the fixed encodings used in Attention is All You Need
# This is because learned encodings seem to be popular in decoder-only models, like GPT-2
# plus, it's simpler to implement in pytorch
# Later, I won't use this class. I just use the nn.Embedding
class PositionalEmbedding(nn.Module):
    def __init__(self, max_position_embeddings, hidden_size):
        super().__init__()
        self.pos_embedding = nn.Embedding(max_position_embeddings, hidden_size)
    
    def forward(self, pos):
        return self.pos_embedding(pos)

## Then implement the function which calculates attention, given (Q,K,V) as arguments.

In [3]:
def attention(q, k, v):
    """
    Args:
        - q: torch.tensor(batch_size, num_heads, seq_len, headsize)
        - k: torch.tensor(batch_size, num_heads, seq_len, headsize)
        - v: torch.tensor(batch_size, num_heads, seq_len, headsize)
    Returns:
        - out: torch.tensor(batch_size, num_heads, seq_len, headsize)
    """
    headsize = q.shape[-1]
    attn_scores = q.matmul(k.transpose(-1, -2)) / math.sqrt(headsize)
    attn_scores = attn_scores.softmax(dim=-1)
    out = attn_scores.matmul(v)
    return out

## Now implement the masking function.

In [4]:
def mask_scores(attn_scores):
    """
    Args:
        attn_scores: torch.tensor of shape (batch_size, num_heads, seq_len, seq_len)
    Returns:
        out: torch.tensor of shape (batch_size, num_heads, seq_len, seq_len)
    """
    seq_len = attn_scores.shape[-2]
    neg_inf = torch.tensor(-1e9).to(attn_scores.device)
    q_ind = torch.arange(seq_len).unsqueeze(1)
    k_ind = torch.arange(seq_len).unsqueeze(0)
    mask = (q_ind < k_ind).to(attn_scores.device)
    attn_scores = torch.where(mask, neg_inf, attn_scores)
    return attn_scores

In [5]:
def masked_attn(q, k, v):
    """
    in:
        - q: torch.tensor of shape (batch_size, num_heads, seq_len, headsize)
        - k: torch.tensor of shape (batch_size, num_heads, seq_len, headsize)
        - v: torch.tensor of shape (batch_size, num_heads, seq_len, headsize)
    out:
        - out: torch.tensor of shape (batch_size, num_heads, seq_len, headsize)
    """
    headsize = q.shape[-1]
    attn_scores = q.matmul(k.transpose(-1, -2)) / math.sqrt(headsize)
    attn_scores = mask_scores(attn_scores)
    attn_scores = attn_scores.softmax(dim=-1)
    out = attn_scores.matmul(v)
    return out

## Put it all together to form an entire attention block.

In [6]:
class MaskedMultiHeadedAttn(nn.Module):
    def __init__(self, num_heads, hidden_size):
        super().__init__()
        self.q_proj = nn.Linear(hidden_size, hidden_size)
        self.k_proj = nn.Linear(hidden_size, hidden_size)
        self.v_proj = nn.Linear(hidden_size, hidden_size)
        self.out_proj = nn.Linear(hidden_size, hidden_size)

        self.num_heads = num_heads
        assert hidden_size % num_heads == 0
        self.headsize = hidden_size // self.num_heads
    
    def forward(self, x):
        """
        in:
            - x: torch.tensor of shape (batch_size, seq_len, hidden_size)
        out:
            - out: torch.tensor of shape (batch_size, seq_len, hidden_size)
        """
        batch_size, seq_len, hidden_size = x.shape
        q = self.q_proj(x).view(batch_size, seq_len, self.num_heads, self.headsize).transpose(1, 2)
        k = self.k_proj(x).view(batch_size, seq_len, self.num_heads, self.headsize).transpose(1, 2)
        v = self.v_proj(x).view(batch_size, seq_len, self.num_heads, self.headsize).transpose(1, 2)
        out = masked_attn(q, k, v)
        out = out.transpose(1, 2).contiguous().view(batch_size, seq_len, -1)
        out = self.out_proj(out)
        return out

## Finish the whole architecture.

In [7]:
class DecoderBlock(nn.Module):
    def __init__(self, num_heads, hidden_size, dropout):
        super().__init__()
        self.attn = MaskedMultiHeadedAttn(num_heads, hidden_size)
        self.ln1 = nn.LayerNorm(hidden_size)
        self.ln2 = nn.LayerNorm(hidden_size)
        self.lin1 = nn.Linear(hidden_size, hidden_size * 4)
        self.lin2 = nn.Linear(hidden_size * 4, hidden_size)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        """
        in:
            - x : torch.tensor of shape (batch_size, seq_len, emb_dim)
        out:
            - out: torch.tensor of shape (batch_size, seq_len, emb_dim)
        """
        x = x + self.attn(self.ln1(x))
        x = x + self.dropout(self.lin2(F.gelu(self.lin1(self.ln2(x)))))
        return x

In [8]:
class DecoderOnlyTransformer(nn.Module):
    def __init__(self, vocab_size, max_pos_embeddings, num_heads, hidden_size, num_layers, dropout):
        super().__init__()
        self.token_embedding = nn.Embedding(vocab_size, hidden_size)
        self.pos_embedding = nn.Embedding(max_pos_embeddings, hidden_size)
        self.dropout = nn.Dropout(dropout)
        self.blocks = nn.Sequential(
            *[
                DecoderBlock(num_heads, hidden_size, dropout)
                for _ in range(num_layers)
            ]
        )
        self.ln = nn.LayerNorm(hidden_size)
        self.lm_head = nn.Linear(hidden_size, vocab_size, bias=False)

    
    def forward(self, input_ids):
        """
        in: 
            - input_ids : torch.tensor of shape (batch_size, seq_len)
        out:
            - logits: torch.tensor of shape (batch_size, seq_len, vocab_size)
        """
        batch_size, seq_len = input_ids.shape
        pos = torch.arange(seq_len).to(input_ids.device)
        x = self.dropout(self.token_embedding(input_ids) + self.pos_embedding(pos))
        x = self.blocks(x)
        x = self.ln(x)
        out = self.lm_head(x)
        return out

## To check you have the attention mask set up correctly, train your model on a toy task, such as reversing a random sequence of tokens. The model should be able to predict the second sequence, but not the first.

In [9]:
class ReverseDataset(Dataset):
    def __init__(self, ndigit):
        self.ndigit = ndigit
        self.vocab_size = 10 # 10 possible digits 0..9
        self.size = 10**self.ndigit # total number of possible combinations

    def __len__(self):
        return self.size

    def __getitem__(self, idx):
        x = torch.randint(self.vocab_size, size=(self.ndigit,), dtype=torch.long)
        y = torch.flip(x,(-1,))
        return x, y

In [10]:
# create a dataset for e.g. 6-digit sequence reversals
ndigit = 6
train_dataset = ReverseDataset(ndigit=ndigit)
train_dataset[0]

(tensor([3, 6, 4, 7, 7, 7]), tensor([7, 7, 7, 4, 6, 3]))

In [11]:
batch_size = 2048
train_loader = DataLoader(
    train_dataset, shuffle=True, pin_memory=True, batch_size=batch_size
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device:", device)

device: cuda


In [12]:
model = DecoderOnlyTransformer(
    num_layers=2,
    num_heads=4,
    vocab_size=train_dataset.vocab_size,
    hidden_size=128,
    max_pos_embeddings=train_dataset.ndigit,
    dropout=0.1,
).to(device).train()

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=6e-4)

max_epochs = 1
for epoch in range(max_epochs):
    pbar = tqdm(enumerate(train_loader), total=len(train_loader))
    for it, (x, y) in pbar:
        x = x.to(device)
        y = y.to(device)

        optimizer.zero_grad()
        
        logits = model(x)
        loss = loss_fn(logits.view(-1, logits.size(-1)), y.view(-1))
        loss.backward()

        optimizer.step()
        pbar.set_description(f"epoch {epoch} iter {it}: train loss {loss.item():.5f}")

epoch 0 iter 488: train loss 1.15316: 100%|██████████| 489/489 [00:25<00:00, 19.29it/s]


In [13]:
# test: notice the first half of predictions are wrong, but the second half are correct
inp = torch.tensor([[1, 2, 3, 4, 5, 6]]).to(device)
logits = model(inp)
print("prediction:", logits.argmax(dim=-1))
print("answer:", torch.flip(inp, (-1,)))

prediction: tensor([[4, 4, 9, 3, 2, 1]], device='cuda:0')
answer: tensor([[6, 5, 4, 3, 2, 1]], device='cuda:0')


## Finally, train your model on the [complete works of William Shakespeare](https://www.gutenberg.org/files/100/100-0.txt). Tokenize the corpus by splitting at word boundaries (re.split(r"\b", ...)).

In [14]:
# you'll need to upload this file to your colab session
text = open('100-0.txt', 'r').read()

In [15]:
class WordDataset(Dataset):
    """
    arrange data and targets so that the first i elements of x
    will be asked to predict the i-th element of y. Notice that
    the eventual language model will actually make block_size
    individual predictions at the same time based on this data,
    so we are being clever and amortizing the cost of the forward
    pass of the network. So for example if block_size is 4, then
    we could e.g. sample a chunk of text "w1 w2 w3 w4 w5", the integers in
    x will correspond to "w1 w2 w3 w4" and in y will be "w2 w3 w4 w5". This will
    then actually "multitask" 4 separate examples at the same time
    in the language model:
    - given just "w1", please predict "w2" as next
    - given "w1 w2" please predict "w3" next
    - given "w1 w2 w3" predict "w4" next
    - given "w1 w2 w3 w4" predict "w5" next
    """
    def __init__(self, data, block_size):
        words = re.split(r"\b", data)
        vocab = sorted(list(set(words)))
        data_size, vocab_size = len(words), len(vocab)
        print('data has %d words, %d unique.' % (data_size, vocab_size))
        
        self.stoi = {word: i for i, word in enumerate(vocab)}
        self.itos = {i: word for i, word in enumerate(vocab)}
        self.block_size = block_size
        self.vocab_size = vocab_size
        self.data = words
    
    def __len__(self):
        return len(self.data) - self.block_size

    def __getitem__(self, idx):
        # grab a chunk of (block_size + 1) characters from the data
        chunk = self.data[idx:idx + self.block_size + 1]
        # encode every word to an integer
        dix = [self.stoi[s] for s in chunk]
        x = torch.tensor(dix[:-1], dtype=torch.long)
        y = torch.tensor(dix[1:], dtype=torch.long)
        return x, y

In [16]:
block_size = 128
train_dataset = WordDataset(text, block_size) 

data has 1987763 words, 34541 unique.


In [17]:
batch_size = 64
train_loader = DataLoader(
    train_dataset, shuffle=True, pin_memory=True, batch_size=batch_size
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device:", device)

device: cuda


Note: this can take hours. I stopped when loss was around 0.5

In [ ]:
model = DecoderOnlyTransformer(
    num_layers=8,
    num_heads=8,
    vocab_size=train_dataset.vocab_size,
    hidden_size=512,
    max_pos_embeddings=train_dataset.block_size,
    dropout=0.1,
).to(device).train()

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=6e-4)

max_epochs = 1
for epoch in range(max_epochs):
    pbar = tqdm(enumerate(train_loader), total=len(train_loader))
    for it, (x, y) in pbar:
        x = x.to(device)
        y = y.to(device)

        optimizer.zero_grad()
        
        logits = model(x)
        loss = loss_fn(logits.view(-1, logits.size(-1)), y.view(-1))
        loss.backward()

        optimizer.step()

        pbar.set_description(f"epoch {epoch} iter {it}: train loss {loss.item():.5f}")

In [19]:
def top_k_logits(logits, k):
    v, ix = torch.topk(logits, k)
    out = logits.clone()
    out[out < v[:, [-1]]] = -float('Inf')
    return out

In [20]:
@torch.no_grad()
def sample(model, x, steps, temperature=1.0, sample=False, top_k=None):
    """
    take a conditioning sequence of indices in x (of shape (b,t)) and predict the next token in
    the sequence, feeding the predictions back into the model each time
    """
    model.eval()
    for k in range(steps):
        x_cond = x if x.size(1) <= block_size else x[:, -block_size:] # crop context if needed
        logits = model(x_cond)
        # pluck the logits at the final step and scale by temperature
        logits = logits[:, -1, :] / temperature
        # optionally crop probabilities to only the top k options
        if top_k is not None:
            logits = top_k_logits(logits, top_k)
        # apply softmax to convert to probabilities
        probs = F.softmax(logits, dim=-1)
        # sample from the distribution or take the most likely
        if sample:
            ix = torch.multinomial(probs, num_samples=1)
        else:
            _, ix = torch.topk(probs, k=1, dim=-1)
        # append to the sequence and continue
        x = torch.cat((x, ix), dim=1)

    return x

In [21]:
context = " O God, O God! "
x = torch.tensor([train_dataset.stoi[s] for s in re.split(r"\b", context)], dtype=torch.long)[None,...].to(device)
y = sample(model, x, 500, temperature=1.0, sample=True, top_k=10)[0]
completion = ''.join([train_dataset.itos[int(i)] for i in y])
print(completion)

 O God, O God! If there be but a shadow in the small
thing more than you should endure the devil than you. You are a good
chorus when ’a is so in the time of Pompey.

WILLIAMS.
I would have you see the King as I may.

PISTOL.
I understand thee not.

FLUELLEN.
I will give you bloody courage off and done; God help me to countenance!
I feel to God his Grace of arms, and I hate’, I stand in Richard’s hall.
O, that we now could come by first the King!

GLOUCESTER.
Never make him welcome with his lady’s words,
When with his body’s right hand this favour now.
This on his part I’ll charm this napkin:
Do but consent; and presently return again,
See when they bid me come my eyes to be.

KING RICHARD.
What I can do I will thankfully be;
For loan oft loses not itself so near.

QUEEN.
As above the slow justice, and in preserve,
With me, and with forgotten head,
With heavy weight and modest breath doth lie,
As the death of this happy womb.

KING RICHARD.
The same of them is too bold and worn,
Unapt 